In [ ]:
from transformers import CLIPVisionModel, CLIPImageProcessor, CLIPVisionConfig

In [6]:
# vision_tower_name = 'openai/clip-vit-large-patch14-336' 
vision_tower_name = 'openai/clip-vit-base-patch32' ## torch.Size([1, 49, 768])
image_processor = CLIPImageProcessor.from_pretrained(vision_tower_name)
vision_tower = CLIPVisionModel.from_pretrained(vision_tower_name)
_ = vision_tower.requires_grad_(False)

vision_tower = vision_tower.to("cuda:1")

In [27]:
def feature_select(image_forward_outs):
    image_features = image_forward_outs.hidden_states[-1] # last layer
    # print(image_features.shape) # 1, 50, 768
    image_features = image_features[:, 1:, :]
    return image_features # 1, 49, 768

## Instruct 150 k coco images --> CLIP embedding

In [28]:
# import pandas as pd
# df=pd.read_json("/media/App/amaranth/lavanya/Capstone_data/llava_instruct_150k.json")
# df.head()

# from datasets import Dataset
# dataset = Dataset.from_pandas(df)

In [29]:
# # Using on sample of COCO 
# import os
# from PIL import Image
# import h5py

# for item in dataset:
#     image_path = f"/media/App/amaranth/lavanya/Capstone_data/train2017/{item['image']}"
#     if os.path.exists(image_path):
#         image = Image.open(image_path)
#         image = image_processor(images=image, return_tensors="pt")
#         image_forward_out = vision_tower(image['pixel_values'].to(device=vision_tower.device), output_hidden_states=True)
#         image_feature = feature_select(image_forward_out)
#         print(image_feature.shape)
#         with h5py.File(f'/media/App/amaranth/lavanya/Capstone_data/clip_features_base_patch32/{os.path.splitext(item["image"])[0]}.h5', 'w') as f:
#             f.create_dataset('image_features', data=image_feature.cpu().numpy())

## Coco train2017 full datatset --> CLIP embeddings

In [30]:
import glob
import os 
from PIL import Image 
import h5py
train_img_names = glob.glob('/media/App/amaranth/lavanya/Capstone_data/train2017/*.jpg')

In [ ]:
for image_path in train_img_names: 
    if os.path.exists(image_path):
        image = Image.open(image_path)
        image = image_processor(images=image, return_tensors="pt")
        image_forward_out = vision_tower(image['pixel_values'].to(device=vision_tower.device), output_hidden_states=True)
        image_feature = feature_select(image_forward_out)
        with h5py.File(f'/media/App/amaranth/lavanya/Capstone_data/clip_features_base_patch32/{os.path.basename(image_path).split(".")[0]}.h5', 'w') as f:
            f.create_dataset('image_features', data=image_feature.cpu().numpy())